# Warehouse demand prediction to optimize inventory

In [1]:
# Importing requierd libraries :

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# Function to calculate the activation function weights ,cost and gradients for the classification model :

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def initialize_weights(n_features):
    weights = np.zeros(n_features)
    bias = 0
    return weights, bias

def compute_cost(y, y_pred):
    m = len(y)
    cost = -(1 / m) * np.sum(y * np.log(y_pred + 1e-10) + (1 - y) * np.log(1 - y_pred + 1e-10))
    return cost

def compute_gradients(X, y, y_pred):
    m = len(y)
    dw = (1 / m) * np.dot(X.T, (y_pred - y))
    db = (1 / m) * np.sum(y_pred - y)
    return dw, db

In [3]:
# Function to train the classification model :

def train_logistic_regression(X, y, learning_rate=0.01, n_iterations=1000):
    n_features = X.shape[1]
    weights, bias = initialize_weights(n_features)
    cost_history = []

    for i in range(n_iterations):
        z = np.dot(X, weights) + bias
        y_pred = sigmoid(z)
        cost = compute_cost(y, y_pred)
        cost_history.append(cost)
        dw, db = compute_gradients(X, y, y_pred)
        weights -= learning_rate * dw
        bias -= learning_rate * db
    return weights, bias, cost_history

In [4]:
# Function to predict the analysis of the classification model :

def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    return np.where(y_pred >= 0.5, 1, 0)

In [5]:
# Function to evaluate the accuracy of the classification model :

def evaluate_model(y_true, y_pred):
    accuracy = np.mean(y_true == y_pred)
    return accuracy

In [16]:
# Loading the data :
data = pd.read_csv('Corrected_Warehouse_Demand_Prediction.csv')
data

,Date,SKU,Product_Category,Previous_Sales,Promotions,Price,Day_of_Week,Region,Lead_Time,Weather,Demand_Level
0,1/1/2023,SKU-464,Electronics,142,No,286.92,Tuesday,East,1,Rainy,Low
1,1/2/2023,SKU-104,Electronics,36,Yes,133.47,Monday,East,9,Sunny,Medium
2,1/3/2023,SKU-218,Food,151,Yes,221.31,Thursday,North,9,Sunny,Medium
3,1/4/2023,SKU-388,Electronics,114,Yes,298.81,Wednesday,North,7,Cloudy,Medium
4,1/5/2023,SKU-473,Electronics,66,No,41.18,Saturday,South,7,Cloudy,High
...,...,...,...,...,...,...,...,...,...,...,...
995,9/22/2025,SKU-191,Furniture,168,No,234.67,Sunday,East,5,Rainy,High
996,9/23/2025,SKU-452,Clothing,142,No,63.76,Saturday,East,4,Sunny,High
997,9/24/2025,SKU-158,Furniture,35,Yes,490.38,Sunday,East,7,Cloudy,Low
998,9/25/2025,SKU-158,Clothing,176,No,111.27,Thursday,North,7,Sunny,Medium


# Data preprocessing :

In [7]:
# Conversion of columns to numerical values(categorical ot binary ) :

data['Previous_Sales'] = pd.to_numeric(data['Previous_Sales'], errors='coerce')
data['Promotions'] = data['Promotions'].map({'Yes': 1, 'No': 0}) 
data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
data['Lead_Time'] = pd.to_numeric(data['Lead_Time'], errors='coerce')

In [8]:
# Feature selction and target variable :

X = data[['Previous_Sales', 'Promotions', 'Price', 'Lead_Time']]
y = data['Demand_Level']

In [9]:
# Encoding variables :

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [10]:
# Dropping out rows with NaN values

X.dropna(inplace=True)
y_encoded = y_encoded[X.index.isin(X.dropna().index)]

C:\Users\AjayG\AppData\Local\Temp\ipykernel_22116\3556303318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


In [11]:
# Splitting training and testing data :

X_train, X_test, y_train, y_test = train_test_split(X.values, y_encoded, test_size=0.2, random_state=42)

In [12]:
# Scaling of table values for dimensional reduction :

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Train the model :

weights, bias, cost_history = train_logistic_regression(X_train_scaled, y_train, learning_rate=0.01, n_iterations=1000)

In [14]:
# Predictions :

y_pred = predict(X_test_scaled, weights, bias)

In [15]:
# Evaluation metrics -Accuracy :

accuracy = evaluate_model(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 34.50%
